In [1]:
%matplotlib inline
import os
import glob
import csv
import re
import numpy as np
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
import pywt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from mpl_toolkits.mplot3d import Axes3D 
from scipy import signal
from scipy.signal import find_peaks
from scipy import optimize
from scipy.stats import linregress, stats
import seaborn as sns

import datetime as dt
from textwrap import wrap


# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
os.getcwd()

'C:\\Users\\bina_\\Dropbox\\Robina Shaheen_PEMS2022\\PEMS_NOx_BC_CO2_ALPR\\ii.dwt'

In [2]:
df1 = pd.read_csv('./Results2/CO2_peakarea_index.csv',)
df2 = pd.read_csv('./Results2/CO2_s6_index_time.csv',)
# df2=df.transpose()
del df1[df1.columns[0]]
print(df1.head())
print(df2.head())
print(len(df1))
print(len(df2))
print(df2.dtypes)

       HW_CO2  peak_height          Ti          Tf      Area_CO2   CO2 (ug/m3)
0  126.247273   368.902634  230.964721  357.211993  46572.951467  83812.264399
1   38.526629   367.884608  406.558750  445.085379  14173.353748  25506.239873
2  113.538243   367.910378  541.237969  654.776212  41771.897940  75172.331671
3   69.130348   376.305601  798.237072  867.367420  26014.137202  46814.807235
4   11.339629   373.332662  928.733648  940.073276   4233.453754   7618.485284
   index  Conc. CO2     time2
0      0  380.262333  0:00:01
1      1  380.194802  0:00:02
2      2  380.116353  0:00:03
3      3  380.028352  0:00:04
4      4  379.931131  0:00:05
486
71054
index           int64
Conc. CO2     float64
time2          object
dtype: object


In [3]:
df1.rename(columns={'Ti':'t1_indexed'}, inplace=True)
# df1.columns
print(df1.dtypes)

HW_CO2         float64
peak_height    float64
t1_indexed     float64
Tf             float64
Area_CO2       float64
CO2 (ug/m3)    float64
dtype: object


In [4]:

df2.rename(columns={'index':'t1_indexed'}, inplace=True)
# df1.columns
print(df2.dtypes)

t1_indexed      int64
Conc. CO2     float64
time2          object
dtype: object


In [5]:
df2.head()

,t1_indexed,Conc. CO2,time2
0,0,380.262333,0:00:01
1,1,380.194802,0:00:02
2,2,380.116353,0:00:03
3,3,380.028352,0:00:04
4,4,379.931131,0:00:05


In [6]:
df1.head()

,HW_CO2,peak_height,t1_indexed,Tf,Area_CO2,CO2 (ug/m3)
0,126.247273,368.902634,230.964721,357.211993,46572.951467,83812.264399
1,38.526629,367.884608,406.558750,445.085379,14173.353748,25506.239873
2,113.538243,367.910378,541.237969,654.776212,41771.897940,75172.331671
3,69.130348,376.305601,798.237072,867.367420,26014.137202,46814.807235
4,11.339629,373.332662,928.733648,940.073276,4233.453754,7618.485284


In [7]:
df1["CO2 EF"] = (df1["CO2 (ug/m3)"]*3.1533)
df1.head()

,HW_CO2,peak_height,t1_indexed,Tf,Area_CO2,CO2 (ug/m3),CO2 EF
0,126.247273,368.902634,230.964721,357.211993,46572.951467,83812.264399,264285.213329
1,38.526629,367.884608,406.558750,445.085379,14173.353748,25506.239873,80428.826192
2,113.538243,367.910378,541.237969,654.776212,41771.897940,75172.331671,237040.913457
3,69.130348,376.305601,798.237072,867.367420,26014.137202,46814.807235,147621.131654
4,11.339629,373.332662,928.733648,940.073276,4233.453754,7618.485284,24023.369645


In [8]:
print(df2.dtypes)
# print(df1.dtypes)

t1_indexed      int64
Conc. CO2     float64
time2          object
dtype: object


In [9]:
df2['t1_indexed'] = df2['t1_indexed'].astype(float)
df2.dtypes

t1_indexed    float64
Conc. CO2     float64
time2          object
dtype: object

In [10]:
df1.sort_values(by='t1_indexed', inplace=True)
df2.sort_values(by='t1_indexed', inplace=True)

In [11]:
df3= pd.merge_asof(
    df1, df2, on="t1_indexed",direction="nearest",  
        allow_exact_matches=False)
print(len(df3))
df3.head()

486


,HW_CO2,peak_height,t1_indexed,Tf,Area_CO2,CO2 (ug/m3),CO2 EF,Conc. CO2,time2
0,126.247273,368.902634,230.964721,357.211993,46572.951467,83812.264399,264285.213329,368.904678,0:05:40
1,38.526629,367.884608,406.558750,445.085379,14173.353748,25506.239873,80428.826192,367.897092,0:08:36
2,113.538243,367.910378,541.237969,654.776212,41771.897940,75172.331671,237040.913457,367.887545,0:10:50
3,69.130348,376.305601,798.237072,867.367420,26014.137202,46814.807235,147621.131654,376.272816,0:15:07
4,11.339629,373.332662,928.733648,940.073276,4233.453754,7618.485284,24023.369645,373.336148,0:17:18


In [12]:
df3.to_csv('./Results2/merged_CO2_EF_index.csv')